# Twitter_Sentiment_Analysis_Random_Forest_Ensemble_Model

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
df = pd.read_csv("/content/Tweet1.csv")

In [7]:
df.head()

,Tweet,Category
0,Not Available,negative
1,IOS 9 App Transport Security. Mm need to check...,neutral
2,"Mar if you have an iOS device, you should down...",neutral
3,@jimmie_vanagon my phone does not run on lates...,negative
4,Not sure how to start your publication on iOS?...,positive


In [8]:
# library to clean data 
import re  
  
# Natural Language Tool Kit 
import nltk  
  
nltk.download('stopwords') 
  
# to remove stopword 
from nltk.corpus import stopwords 
  
# for Stemming propose  
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# # Building Cleaned corpus by removing stop word, stemming and normalising 

In [0]:
corpus = []  
  
# 1000 (reviews) rows to clean 
for i in range(0, 5967):  
      
    # column : "Review", row ith 
    Tweet = re.sub('[^a-zA-Z]', ' ', df['Tweet'][i])  
      
    # convert all cases to lower cases 
    Tweet = Tweet.lower()  
      
    # split to array(default delimiter is " ") 
    Tweet = Tweet.split()  
      
    # creating PorterStemmer object to 
    # take main stem of each word 
    ps = PorterStemmer()  
    # loop for stemming each word 
    # in string array at ith row     
    Tweet = [ps.stem(word) for word in Tweet 
                if not word in set(stopwords.words('english'))]  
                  
    # rejoin all string array elements 
    # to create back into a string 
    Tweet = ' '.join(Tweet)   
      
    # append each string to create 
    # array of clean text  
    corpus.append(Tweet)  

In [11]:
corpus

['avail',
 'io app transport secur mm need check rd parti network pod support http co fmtcfuadgj',
 'mar io devic download app http co gl tn udnd',
 'jimmi vanagon phone run latest io may account problem day time replac',
 'sure start public io live help ask anyth session today friday http co kpqqgjjh x',
 'two dollar tuesday forklift quickkey io suit page today http co bnmfoeacw',
 'alreadi sign test io game now chanc http co ezhiu edpq',
 'youtub game offici launch web android io august youtub final go r http co qbhya kawp webseri',
 'youtub game launch tomorrow io android app go head head twitch http co yzoattoejc io game',
 'astril yashan bbc bbcchines vpn access io may limit china militari parad chanc chat',
 'parkrun app io download great app easier access info amp readi saturday run http co zyofhzerio',
 'today youtubegam launch app io android devic us uk need know http co kf dgnhx b',
 'got project want work need help swift need excus hang io dev sat hang us http co puwskbdfw',

# # Creating Bag words using CountVectorizer

In [0]:
# Creating the Bag of Words model 
from sklearn.feature_extraction.text import CountVectorizer 
  
# To extract max 1500 feature. 
# "max_features" is attribute to 
# experiment with to get better results 
cv = CountVectorizer(max_features = 10500)  
  
# X contains corpus (dependent variable) 
X = cv.fit_transform(corpus).toarray()  
  
# y contains answers if review 
# is positive or negative 
y = df.iloc[:, 1].values  

In [14]:
X.shape
y.shape

(5967,)

# Train snd Test Split

In [0]:
# Splitting the dataset into 
# the Training set and Test set 
from sklearn.model_selection import train_test_split 
  
# experiment with "test_size" 
# to get better results 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25) 

In [0]:
n_estimators = [10,500,1000]
criterion = ['gini', 'entropy']
min_samples_split=[2,10,20,50,100]
max_depth=[20,30,40,50]

#  Finding best parametrs using Randomized Search and Fit the model

In [0]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import RandomizedSearchCV
  
# n_estimators can be said as number of 
# trees, experiment with n_estimators 
# to get better results  
#model = RandomForestClassifier(n_estimators = 300, criterion = 'entropy') 
                              
#model.fit(X_train, y_train) 

In [18]:
param_grid = { 'n_estimators' : n_estimators,
             'criterion' : criterion , 'min_samples_split' : min_samples_split , 'max_depth':max_depth}
rf = RandomForestClassifier()
rf_g= RandomizedSearchCV(rf, param_grid, cv=5, verbose=2 , n_jobs=-1)
rf_g.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 41.5min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

rf_g.best_estimator_

# # Prediction and Accuracy

In [0]:
predicted = rf_g.predict(X_test)

In [0]:
from sklearn import metrics

In [22]:
metrics.accuracy_score(predicted,y_test)

0.5442359249329759

In [23]:
print(metrics.classification_report(predicted,y_test))

              precision    recall  f1-score   support

    negative       0.09      0.61      0.15        33
     neutral       0.20      0.48      0.28       229
    positive       0.95      0.56      0.70      1230

    accuracy                           0.54      1492
   macro avg       0.41      0.55      0.38      1492
weighted avg       0.81      0.54      0.62      1492



In [26]:
plt.figure(figsize=(8,4))
print('Confusion MAtrix')

Confusion MAtrix


<Figure size 576x288 with 0 Axes>

In [32]:
cf =  metrics.confusion_matrix(predicted,y_test)
cf

array([[ 20,   9,   4],
       [ 85, 109,  35],
       [124, 423, 683]])